In [118]:
import pandas as pd
import itertools
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import ast
import seaborn as sns
import os
import datetime

In [119]:
# open classes.xlsx as classes
classes = pd.read_excel('classes.xlsx', index_col=0)

# open schedules.xlsx as schedules
schedules = pd.read_excel('schedules.xlsx')
schedules['Course'] = schedules['Course'].astype(str)
schedules['Days'] = schedules['Days'].apply(ast.literal_eval)

In [120]:
# create a list of all unique courses by concatenating Department and Course in the classes df
courses = []
for row in classes.itertuples():
    courses.append({'Department': row.Department, 'Course': str(row.Course)})

In [121]:
def generate_combos(courses, num_courses=6, must_haves=[], at_least=[], must_select=[], at_most=[], unique_deps=True):
    # get all possible combinations of six courses
    combos = list(itertools.combinations(courses, num_courses))

    if unique_deps:
        # eliminate schedules with two or more tuples where the Department is the same
        combos = [tpl for tpl in combos if len(set(course['Department'] for course in tpl)) == num_courses]

    for must_have in must_haves:
        combos = [tpl for tpl in combos if any((must_have in (course['Department'] + course['Course'])) for course in tpl)]

    for least in at_least:
        combos = [tpl for tpl in combos if sum(any(e in (course['Department'] + course['Course']) for e in least['Courses']) for course in tpl) >= least['Number']]

    for select in must_select:
        combos = [tpl for tpl in combos if sum(any(e in (course['Department'] + course['Course']) for e in select['Courses']) for course in tpl) == select['Number']]

    for most in at_most:
        combos = [tpl for tpl in combos if sum(any(e in (course['Department'] + course['Course']) for e in most['Courses']) for course in tpl) <= most['Number']]

    return combos

# params = {'must_haves': ['MAT203', 'PHI385', 'CHI', 'SPA'], 
#           'at_least': [{'Courses': ['CHI', 'SPA'], 'Number': 1}], 
#           'at_most':[{'Courses': ['CHI', 'SPA', 'FRE', "RUS", "JPN", "KOR", "PLS", "ASL"], 'Number': 2}]}

# params = {'num_courses': 6, 'must_haves': ['MAT', 'PHI385', 'CHI', 'SPA', 'CWR', 'COS'],
#           'at_most':[{'Courses': ['CHI', 'SPA', 'FRE', "RUS", "JPN", "KOR", "PLS", "ASL"], 'Number': 2}]}

params = {'num_courses': 6,
          'must_haves': ['MAT', 'PHI385', 'CHI', 'WRI'],
          'at_least': [{'Courses': ['PSY', 'CWR', 'SPI', 'SPA', 'ECO'], 'Number': 2}],
           'at_most':[{'Courses': ['CHI', 'SPA', 'FRE', "RUS", "JPN", "KOR", "PLS", "ASL"], 'Number': 2}]
        }

combos = generate_combos(courses, **params)

params['day_limits'] = {}

len(combos)

36

In [122]:
schedules['Key'] = schedules['Department'] + schedules['Course']

results = []

for combo in combos:
    courses = pd.DataFrame(list(combo))
    courses['Key'] = courses['Department'] + courses['Course']
    
    schedule = schedules[schedules['Key'].isin(courses['Key'])]
    
    if len(courses['Key'].unique()) != len(schedule['Key'].unique()):
        continue

    schedule['Section'] = schedule['Section'].str[0]

    schedule['Key'] = schedule['Department'] + schedule['Course'] + schedule['Section']

    schedule.drop(columns=['Department', 'Course', 'Section'], inplace=True)

    key_counts = schedule['Key'].value_counts().reset_index()
    key_counts.columns = ['Key', 'Count']
    schedule = schedule.merge(key_counts, on='Key')
    schedule = schedule.sort_values(by=['Count'])

    def check_overlap(existing_times, new_times):
        for row in existing_times:
            if any([day in row['Days'] for day in new_times['Days']]) and (row['End'] >= new_times['Start']) and (row['Start'] <= new_times['End']):
                return False
        return True

    def find_combination(schedule, keys, current_schedule):
        key = keys[0]

        # get rows of schedule where Key is key as a df called key_df
        key_df = schedule[schedule['Key'] == key]

        # iterate through rows of key_df
        for index, row in key_df.iterrows():
            # check if row overlaps with any rows in current_schedule
            if check_overlap(current_schedule, row):
                # if not, add row to current_schedule
                new_schedule = current_schedule.copy()
                new_schedule.append(row)
                # if keys is empty, return current_schedule
                if len(keys) == 1:
                    return new_schedule
                # else, call find_combination with keys[1:], current_schedule
                else:
                    result = find_combination(schedule, keys[1:], new_schedule)
                    if result:
                        return result

        return None

    # get unique keys
    keys = schedule['Key'].unique()

    result = find_combination(schedule, keys, [])
    if result:
        results.append(pd.DataFrame(result))

len(results)

C:\Users\kguan17\AppData\Local\Temp\ipykernel_16804\4257785546.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schedule['Section'] = schedule['Section'].str[0]
C:\Users\kguan17\AppData\Local\Temp\ipykernel_16804\4257785546.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schedule['Key'] = schedule['Department'] + schedule['Course'] + schedule['Section']
C:\Users\kguan17\AppData\Local\Temp\ipykernel_16804\4257785546.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

8

In [123]:
filtered_results = []

for df in results:
    day_limits_copy = params['day_limits'].copy()  # Create a copy of day_limits for each dataframe
    exceed_limit = False
    
    for index, row in df.iterrows():
        days_list = row['Days']
        
        for day in days_list:
            if day in day_limits_copy.keys():
                if day_limits_copy[day] <= 0:
                    exceed_limit = True
                    break
                day_limits_copy[day] -= 1
        
        if exceed_limit:
            break
    
    if not exceed_limit:
        filtered_results.append(df)

results = filtered_results

len(results)

8

In [124]:
if len(results) != 0:
    current_datetime = datetime.datetime.now()
    formatted_datetime = current_datetime.strftime("%d_%m_%y_%H_%M_%S")
    directory_name = f"schedules_{formatted_datetime}"

    # Create the directory in the current working directory
    try:
        os.mkdir(directory_name)
        print(f"Directory '{directory_name}' created successfully.")
    except OSError:
        print(f"Creation of directory '{directory_name}' failed.")

    def get_text_dimensions(text_string, font):
        # https://stackoverflow.com/a/46220683/9263761
        ascent, descent = font.getmetrics()

        text_width = font.getmask(text_string).getbbox()[2]
        text_height = font.getmask(text_string).getbbox()[3] + descent

        return text_width, text_height

    for i, result in enumerate(results):
        schedule = result

        # Convert start and end times to datetime objects
        schedule['Start'] = pd.to_datetime(schedule['Start'])
        schedule['End'] = pd.to_datetime(schedule['End'])
        
        # generate a list of pastel colors
        classes = [x[:-1] for x in list(schedule['Key'].unique())]
        palette = sns.color_palette('pastel', len(classes)).as_hex()

        colors = {}
        for j, c in enumerate(classes):
            colors[c] = palette[j]

        # Create a blank image with the desired size
        width = 3000
        height = 2400
        image = Image.new('RGB', (width, height), 'white')
        draw = ImageDraw.Draw(image)

        time_font = ImageFont.truetype('arial.ttf', 40)
        day_font = ImageFont.truetype('arial.ttf', 60)
        class_font = ImageFont.truetype('arial.ttf', 36)

        # Define the width and height of each time slot
        time_slot_width = width // 5
        time_slot_height = height // 24

        # Draw day labels on top
        for day_idx, day in enumerate(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']):
            text_width, text_height = get_text_dimensions(day, font=day_font)
            x = day_idx * time_slot_width + (time_slot_width - text_width) // 2
            y = 5  # Some padding
            draw.text((x, y), day, font=day_font, fill='black')

        # Draw the time slots for each day and draw time labels on the left
        for day_idx, day in enumerate(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']):
            for hour in range(24):
                top_left = (day_idx * time_slot_width, hour * time_slot_height)
                bottom_right = ((day_idx + 1) * time_slot_width, (hour + 1) * time_slot_height)
                draw.rectangle([top_left, bottom_right], outline='black')

        # Draw the course schedule
        for idx, row in schedule.iterrows():
            key = row['Key']
            days = row['Days']
            start_hour = row['Start'].hour
            start_minute = row['Start'].minute
            end_hour = row['End'].hour
            end_minute = row['End'].minute
            color = colors[key[:-1]]

            for day in days:
                day_idx = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'].index(day)
                top_left = (day_idx * time_slot_width, start_hour * time_slot_height + (start_minute / 60) * time_slot_height)
                bottom_right = (day_idx * time_slot_width + time_slot_width, end_hour * time_slot_height + (end_minute / 60) * time_slot_height)
                draw.rectangle([top_left, bottom_right], fill=color)

                # Draw class labels (Key)
                class_label = f"{key} ({start_hour:02}:{start_minute:02} - {end_hour:02}:{end_minute:02})"
                text_width, text_height = get_text_dimensions(class_label, font=class_font)
                x = day_idx * time_slot_width + (time_slot_width - text_width) // 2
                y = start_hour * time_slot_height + (start_minute / 60) * time_slot_height + (time_slot_height - text_height) // 2
                draw.text((x, y), class_label, font=class_font, fill='black')

        extra_width = int(width * 0.05)

        # Create a new image with added blank space on the left
        new_width = width + extra_width
        new_image = Image.new('RGB', (new_width, height), 'white')

        # Paste the original image onto the new image, shifted to the right
        new_image.paste(image, (extra_width, 0))

        image = new_image
        draw = ImageDraw.Draw(image)

        for hour in range(24):
            text = f'{hour:02}:00'
            text_width, text_height = get_text_dimensions(text, font=time_font)
            x = 5  # Some padding
            y = hour * time_slot_height + (time_slot_height - text_height) // 2
            draw.text((x, y), text, font=time_font, fill='black')

        filename = f"course_schedule{i}.png"
        image_path = os.path.join(directory_name, filename)
        image.save(image_path)

    # Write the dictionary to a text file
    params_filename = "params.txt"
    params_file_path = os.path.join(directory_name, params_filename)

    with open(params_file_path, "w") as params_file:
        for key, value in params.items():
            params_file.write(f"{key}: {value}\n")
else:
    print('No valid schedules.')

Directory 'schedules_08_08_23_12_19_35' created successfully.


C:\Users\kguan17\AppData\Local\Temp\ipykernel_16804\774756401.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schedule['Start'] = pd.to_datetime(schedule['Start'])
C:\Users\kguan17\AppData\Local\Temp\ipykernel_16804\774756401.py:27: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schedule['End'] = pd.to_datetime(schedule['End'])
C:\Users\kguan17\AppData\Local\Temp\ipykernel_16804\774756401.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schedule['Start'] = pd.to_datetime(schedule['Start'])
C:\Users\kguan17\AppData\Local\Temp\ipykernel_16804\774756401.py:27: UserWarn